<a href="https://www.kaggle.com/code/dewanrapa/senti-ana-splitting-10k-w-bert?scriptVersionId=202707898" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.

In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [6]:
apps_rvw='https://raw.githubusercontent.com/amanullahshah32/Review-Scraping/refs/heads/main/Dataset/all_app_reviews.csv'
df=pd.read_csv(apps_rvw)

In [7]:
df.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code,app_name
0,Google Play,d33f5fbd-e904-4c35-a29d-9bf66d393aa1,A Google user,NaN,nice,5,0,2024-10-10 12:50:00,NaN,NaN,NaN,en,us,com.twgbd.dims
1,Google Play,b1c90db8-c97e-401a-87db-f970ad5b7ce3,A Google user,NaN,good,5,0,2024-10-10 12:43:03,NaN,NaN,2.1.8,en,us,com.twgbd.dims
2,Google Play,fd85498c-702e-4bc7-8cde-70411670e214,A Google user,NaN,good,5,0,2024-10-10 12:35:00,NaN,NaN,2.1.8,en,us,com.twgbd.dims
3,Google Play,b8cc25c1-40c1-46d9-a238-bf0116de6ac4,A Google user,NaN,good,5,0,2024-10-10 12:18:52,NaN,NaN,2.1.8,en,us,com.twgbd.dims
4,Google Play,c0dfc093-493f-4906-80cb-1de222fbb02b,A Google user,NaN,good 😊,4,0,2024-10-10 11:39:51,NaN,NaN,2.1.8,en,us,com.twgbd.dims


In [8]:
df.shape

(99395, 14)

In [9]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code,app_name
0,Google Play,f6c1ce97-8e1a-436f-b5bd-327babc72c88,A Google user,NaN,Easy to use and I like the reminder.,5,0,2023-03-05 14:13:00,NaN,NaN,8.5.1,en,us,com.lbrc.PeriodCalendar
1,Google Play,daa1ddec-847a-4bfc-a777-7802e6144e01,A Google user,NaN,it is not accurate,3,0,2019-06-22 07:30:46,Hi Ntombi!\nThank you for the feedback. To get...,2019-06-24 19:42:53,6.1.1,en,us,com.lbrc.PeriodCalendar
2,Google Play,1fd38dc0-938a-4174-bf84-17ae4e32a381,Mohammad Nasruddin,NaN,md. gulfam hi,5,0,2023-07-09 11:40:57,NaN,NaN,12.4.5,en,us,com.ronasoftstudios.earmaxfxpro
3,Google Play,bc492cbc-3422-4469-b3a5-7807d10b4cdd,Vaso Gelashvili,NaN,Oxford? Xd ...,1,1,2023-06-04 22:38:26,NaN,NaN,11.1.544,en,us,com.mobisystems.msdict.embedded.wireless.oxfor...
4,Google Play,e2fd4183-b642-4c48-aa31-6db87b1cd3d9,A Google user,NaN,Fab.. I always need to write down usually coz ...,5,35,2021-06-12 17:39:32,NaN,NaN,7.8.3,en,us,com.lbrc.PeriodCalendar


In [10]:
df_sampled = df_shuffled.sample(n=10000, random_state=42)

In [11]:
df_sampled.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code,app_name
43883,Google Play,ece936cd-ce1a-468c-9383-f9917822ac10,Maruf Hossain,NaN,It crush many time. Don't work properly Server...,2,3,2024-02-08 02:43:58,NaN,NaN,1.0.1,en,us,com.medexplus.digitalitsheba.app
21124,Google Play,5d3244ab-72aa-4be9-ada5-6cc373a40ef0,A Google user,NaN,Very good Apps,5,0,2022-11-25 08:23:05,NaN,NaN,2.0.17,en,us,com.twgbd.dims
29732,Google Play,ff9ecbe4-7e9e-454c-b709-1f461787d5f2,A Google user,NaN,Nice,5,0,2022-03-26 16:10:01,Thank you so much for your feedback.,2022-03-27 05:28:40,2.0.12,en,us,com.twgbd.dims
88813,Google Play,7a3ea251-e533-4179-80ff-fb846986d7ed,A Google user,NaN,I love it,5,0,2020-11-25 15:06:34,NaN,NaN,7.5.1,en,us,com.lbrc.PeriodCalendar
28576,Google Play,820cb1b6-d359-4f27-8870-902a52899b30,A Google user,NaN,Uniomcode,5,2,2019-10-19 08:14:09,NaN,NaN,NaN,en,us,com.msd.consumer


In [12]:
df_sampled.shape

(10000, 14)

In [13]:
df_review_description = df_sampled[['review_description']]

In [14]:
df_review_description.head()

,review_description
43883,It crush many time. Don't work properly Server...
21124,Very good Apps
29732,Nice
88813,I love it
28576,Uniomcode


In [15]:
df_review_description.shape

(10000, 1)

In [16]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Load your 'df_review_description' dataset
# If you saved it to CSV and want to reload:
# df_review_description = pd.read_csv('review_description_only.csv')

# Step 1: Convert the 'review_description' column to strings and handle NaN values
df_review_description['review_description'] = df_review_description['review_description'].astype(str).fillna('')

# Step 2: Load pre-trained BERT model for sentiment analysis
# You can use a model fine-tuned for sentiment analysis like 'nlptown/bert-base-multilingual-uncased-sentiment'
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Step 3: Use Hugging Face's pipeline for sentiment analysis
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Step 4: Apply sentiment analysis to the 'review_description' column
# Only apply sentiment analysis to non-empty strings and store both the label and score
def get_sentiment_score(text):
    if isinstance(text, str) and text.strip():
        result = classifier(text)[0]
        return result['label'], result['score']  # Return both sentiment label and score
    return 'neutral', 0  # Default sentiment for empty text

# Apply the BERT sentiment analysis and create two new columns: 'sentiment_label' and 'sentiment_score'
df_review_description['sentiment_label'], df_review_description['sentiment_score'] = zip(*df_review_description['review_description'].apply(get_sentiment_score))

# Step 5: Display the DataFrame with 'review_description' and 'sentiment_score'
df_review_description[['review_description', 'sentiment_score']]


/tmp/ipykernel_23/3144448547.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_description['review_description'] = df_review_description['review_description'].astype(str).fillna('')
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/tmp/ipykernel_23/3144448547.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_description['sentiment_label'], df_review_description['sentiment_score'] = zip(

,review_description,sentiment_score
43883,It crush many time. Don't work properly Server...,0.676273
21124,Very good Apps,0.612561
29732,Nice,0.436841
88813,I love it,0.833338
28576,Uniomcode,0.302239
...,...,...
44546,This app is really helpful and easy to use! Ju...,0.799761
72889,"It's accurate, I can keep track of my next period",0.550626
8781,"It's is really good, but I only had it for thr...",0.451962
36752,I like it,0.518116
